# 유튜브 유사 채널 탐색

이 코드는 유튜브 채널 데이터를 기반으로 각 채널의 키워드를 임베딩한 후, **코사인 유사도**를 사용하여 유사한 채널을 탐색하는 과정을 설명합니다. **Word2Vec** 임베딩 기법을 적용하여 채널의 특성을 나타내는 키워드 간의 의미적 관계를 학습하고, 이를 바탕으로 채널 간의 유사성을 정량적으로 평가합니다. 

이 분석 과정은 콘텐츠 제작자에게 유사한 주제를 다루는 채널을 발견하게 하여, 협업, 마케팅 전략, 또는 콘텐츠 방향 설정에 중요한 인사이트를 제공합니다.

## 주요 단계:
1. **키워드 임베딩**: 각 채널의 키워드를 Word2Vec을 통해 벡터로 변환.
2. **유사도 계산**: 코사인 유사도를 사용하여 채널 간 유사성 측정.
3. **유사 채널 탐색**: 분석하려는 채널과 키워드가 유사한 채널을 우선적으로 검토하여, 보다 정교한 결과를 도출.
4. **결과 분석**: 유사한 채널 리스트와 그 채널들이 가진 콘텐츠 특징을 분석하여 전략적 인사이트 제공.

이를 통해 채널 간의 관계를 효율적으로 파악하고, 콘텐츠 전략 수립에 중요한 참고 자료로 활용할 수 있습니다.


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec

In [2]:
# 구독자 수에 따라 범위 클러스터를 반환하는 함수
def subscribers_range(subscribers):
    # 구독자가 1000 미만인 경우 0을 반환
    if subscribers < 1000:
        return 0
    # 구독자가 1000 이상 10,000 미만인 경우 1을 반환
    elif subscribers >= 1000 and subscribers < 10000:
        return 1
    # 구독자가 10,000 이상 50,000 미만인 경우 2를 반환
    elif subscribers >= 10000 and subscribers < 50000:
        return 2
    # 구독자가 50,000 이상 100,000 미만인 경우 3을 반환
    elif subscribers >= 50000 and subscribers < 100000:
        return 3
    # 구독자가 100,000 이상 500,000 미만인 경우 4를 반환
    elif subscribers >= 100000 and subscribers < 500000:
        return 4
    # 구독자가 500,000 이상 1,000,000 미만인 경우 5를 반환
    elif subscribers >= 500000 and subscribers < 1000000:
        return 5
    # 구독자가 1,000,000 이상 5,000,000 미만인 경우 6을 반환
    elif subscribers >= 1000000 and subscribers < 5000000:
        return 6
    # 구독자가 5,000,000 이상 10,000,000 미만인 경우 7을 반환
    elif subscribers >= 5000000 and subscribers < 10000000:
        return 7
    # 구독자가 10,000,000 이상인 경우 8을 반환
    else:
        return 8


In [3]:
import pandas as pd

# 채널 데이터 불러오기
df = pd.read_csv("../data/channel_sample_2790.csv", encoding="utf-8-sig", index_col=0).sort_index().reset_index(drop=True)

# 각 채널의 구독자 수를 기반으로 구독자 범위 클러스터를 계산하여 새로운 열에 추가
df["subscribers_cluster"] = df.channel_subscribers.apply(lambda x: subscribers_range(x))

# 'MAINLY_USED_KEYWORDS' 열의 각 문자열을 리스트로 변환하고, 요소의 공백을 제거
df["MAINLY_USED_KEYWORDS"] = df.MAINLY_USED_KEYWORDS.apply(lambda x: [word.strip() for word in x.split(",")])

# 'MAINLY_USED_TAGS' 열의 각 문자열을 리스트로 변환하고, 요소의 공백을 제거
df["MAINLY_USED_TAGS"] = df.MAINLY_USED_TAGS.apply(lambda x: [word.strip() for word in x.split(",")])

df.head()


,CHANNEL_ID,CHANNEL_NAME,CHANNEL_DESCRIPTION,CHANNEL_TAGS,MAINLY_USED_KEYWORDS,MAINLY_USED_TAGS,CHANNEL_COUNTRY,CHANNEL_LINK,CHANNEL_SINCE,CHANNEL_CLUSTER,CRAWLED_DATE,USER_ID,channel_id_part,CHANNEL_THUMBNAIL,channel_subscribers,subscribers_cluster
0,UC-_XNR4vMF_he7IRH3zlp5A,SPELLA,"?? ICONIC 스펠라\nBased in Seoul, South Korea.\nW...",NaN,"[텃팅, DANCE, 스펠라, 터팅, TUTTING]","[스펠라, HANDS, 터팅, MEGAN, THEE]",대한민국,NaN,2013-10-20,55,2024-10-13 17:00:19,0.0,_,https://yt3.googleusercontent.com/7hoar3IodnuH...,20299,2
1,UC1_KTHHaUcWFdlMmy7zpoug,스트리머_카페인,본진: https://www.youtube.com/carfeine,"['gta온라인', '업데이트', 'gta5', '신규차량', 'gta5신차', '...","[gta5, 레이스하이라이트, 그랑프리, 사이버펑크]","[cyberpunk, gyberpunk, 사이버펑크, 엣지러너]",대한민국,"https://www.youtube.com/channel/carfeine2,http...",2021-05-12,45,2024-10-13 17:00:19,0.0,_,https://yt3.googleusercontent.com/NY9McgAlNdRu...,1880,1
2,UC1_uDngV0wfSnp8ZpsTV4WQ,배저씨,배구아저씨(유소년배구코치)\n\n제이현인터내셔널\n\nhttps://smartsto...,NaN,"[공주, volleyball, 계룡배구클럽, 배구, 결승전]","[공주, volleyball, 계룡배구클럽, 배구, 결승전]",대한민국,NaN,2016-10-12,74,2024-10-13 17:00:19,0.0,_,https://yt3.googleusercontent.com/ytc/AIdro_nc...,1890,1
3,UC2_VzJvcHmGce_xH3PFLjMg,가비TV l 이근갑국어,"2021 수능 성공을 위한 최고의 솔루션, 국어만큼은 이근갑\n낯선 지문도 스스로 ...",NaN,[이근갑],[과학],대한민국,gabistudy.com,2019-12-16,22,2024-10-13 17:00:19,0.0,_,https://yt3.googleusercontent.com/ytc/AIdro_l1...,3140,1
4,UC3_FgiqHMS6uzTVCFCF09qg,백수묵시록,더빙 전문 채널입니다.매주 월요일과 목요일을 제외한 모든 시간.오후 5시~6시 트위...,"['해병문학', '황근출', '톤톤정', 'mcu', '무모칠', '해병', '해병...","[백묵보플, 백묵만화더빙, 백묵해병, 백묵더빙, 해병]","[백묵더빙, 백묵보플, 백묵해병, 백묵만화더빙, 백묵해병더빙]",대한민국,https://gallog.dcinside.com/dhvnsqpxk2/guestbo...,2013-11-29,11,2024-10-13 17:00:19,0.0,_,https://yt3.googleusercontent.com/zsBuKvxigH8I...,16200,2


In [4]:

# 주어진 점수 목록을 지정된 범위로 스케일링하는 함수
def scale_scores(dict_list, new_min=0.1, new_max=0.9):
    # dict_list에서 각 항목의 'score' 값을 추출하여 리스트로 생성
    scores = [d['score'] for d in dict_list]
    
    # 점수의 최소값과 최대값을 계산
    min_score = min(scores)
    max_score = max(scores)
    
    # 각 항목의 점수를 새로운 범위로 스케일링
    for d in dict_list:
        # 스케일링 공식에 따라 새로운 점수 계산
        scaled_value = ((d['score'] - min_score) / (max_score - min_score)) * (new_max - new_min) + new_min
        
        # 클리핑 작업으로 값이 new_min과 new_max 사이에 있도록 제한
        d['score'] = max(min(scaled_value, new_max), new_min)
    
    return dict_list

# 주어진 문장의 단어 벡터를 평균하여 문장 벡터를 생성하는 함수
def get_sentence_vector(sentence, model):
    # 문장에서 모델에 존재하는 단어의 벡터를 가져옴
    vectors = [model.wv[word] for word in sentence if word in model.wv]
    
    # 벡터의 평균을 계산하여 문장 벡터 반환
    return np.mean(vectors, axis=0)


In [12]:
# 채널 데이터에서 랜덤으로 하나의 채널을 샘플링
target_channel = df.sample(n=1)

# 선택된 채널의 CHANNEL_ID를 추출
channel_id = target_channel.CHANNEL_ID.iloc[0]

# 선택된 채널의 주로 사용된 키워드와 태그를 추출
mainly_used_keywords = target_channel.MAINLY_USED_KEYWORDS.iloc[0]
mainly_used_tags = target_channel.MAINLY_USED_TAGS.iloc[0]

# 주로 사용된 키워드와 태그를 합쳐서 중복 없이 리스트로 생성
used_keywords = list(set(mainly_used_keywords) | set(mainly_used_tags))

# 선택된 채널의 클러스터 정보 추출
channel_cluster = target_channel.CHANNEL_CLUSTER.iloc[0]

# 선택된 채널의 구독자 수 추출
channel_subscribers = target_channel.channel_subscribers.iloc[0]
target_channel

,CHANNEL_ID,CHANNEL_NAME,CHANNEL_DESCRIPTION,CHANNEL_TAGS,MAINLY_USED_KEYWORDS,MAINLY_USED_TAGS,CHANNEL_COUNTRY,CHANNEL_LINK,CHANNEL_SINCE,CHANNEL_CLUSTER,CRAWLED_DATE,USER_ID,channel_id_part,CHANNEL_THUMBNAIL,channel_subscribers,subscribers_cluster
2048,UCFQ8D7RnvA1_1BWtIJXu86w,블라인드_Car Story,◦비즈니스 문의 : chomanyeon@gmail.com◦인스타그램 : blind_...,"['할인', '하이브리드', 'bmw', '그랜저', 'x3할인', 'x3', '2...","[하이브리드, 스포티지, 할부, 신차, 최대]","[구매, 할부, BMW, EV3, 하이브리드]",대한민국,"https://www.instagram.com/blind_90,https://bli...",2019-07-08,45,2024-10-13 17:00:19,0.0,q,https://yt3.googleusercontent.com/WbPT8Snqb2P9...,271000,4


In [13]:
# 구독자 수를 기반으로 해당 채널의 구독자 클러스터를 계산
subscribers_cluster = subscribers_range(channel_subscribers)

# 채널 클러스터와 구독자 클러스터가 일치하는 채널을 필터링
# 현재 채널의 구독자 클러스터와 +/- 1의 구독자 클러스터를 가진 채널을 포함
filtered_channels = df[(df.CHANNEL_CLUSTER == channel_cluster) & 
                       (df.subscribers_cluster.isin([subscribers_cluster+1, subscribers_cluster, subscribers_cluster-1]))].copy()

# 필터링된 채널의 주로 사용된 키워드와 태그를 합쳐서 'target_keywords'라는 새 열을 생성
filtered_channels["target_keywords"] = filtered_channels.apply(lambda x: x.MAINLY_USED_KEYWORDS + x.MAINLY_USED_TAGS, axis=1)

filtered_channels.reset_index(drop=True, inplace=True)


In [14]:
### 1) 포함 여부만으로 값 찾기
# 사용된 키워드가 'target_keywords'에 포함된 채널을 필터링하여 first_results DataFrame 생성
first_results = filtered_channels[filtered_channels.apply(
    lambda row: any(keyword in row.target_keywords for keyword in used_keywords), axis=1)]

# 현재 선택된 채널의 ID와 일치하는 채널은 제외
first_results = first_results[(first_results.CHANNEL_ID != channel_id)]

# DataFrame을 복사하여 first_results 변수에 저장
first_results = first_results.copy()

# 'target_keywords'에 포함된 사용된 키워드 수에 따라 점수를 계산하여 'score' 열에 저장
# 점수 계산 공식: 1 + (교집합의 크기 / 합집합의 크기)
first_results['score'] = first_results['target_keywords'].apply(
    lambda x: 1 + (len(set(used_keywords) & set(x))) / (len(set(used_keywords) | set(x)))
)
first_results

,CHANNEL_ID,CHANNEL_NAME,CHANNEL_DESCRIPTION,CHANNEL_TAGS,MAINLY_USED_KEYWORDS,MAINLY_USED_TAGS,CHANNEL_COUNTRY,CHANNEL_LINK,CHANNEL_SINCE,CHANNEL_CLUSTER,CRAWLED_DATE,USER_ID,channel_id_part,CHANNEL_THUMBNAIL,channel_subscribers,subscribers_cluster,target_keywords,score
6,UCrOl7IuctoHW0iW-Gozj4EQ,비피디 BPD,비피디와 솔님의 자동차 이야기광고 마케팅 문의: bpdbpd.biz@gmail.com,"['솔님', '비피디', '자동차', '시승기', '비린내PD', 'VEHICLE'...","[솔님, 전기차, 시승기, 싼타페, 하이브리드]","[솔님, 하이브리드, 시승기, 싼타페, 전기차]",대한민국,"https://www.instagram.com/bpd.pd/,https://www....",2012-07-27,45,2024-10-13 17:00:19,0.0,o,https://yt3.googleusercontent.com/ytc/AIdro_le...,348000,4,"[솔님, 전기차, 시승기, 싼타페, 하이브리드, 솔님, 하이브리드, 시승기, 싼타페...",1.083333


In [15]:
### 2) word2vec 기반의 임베딩으로 값 찾기
# 필터링된 채널의 'target_keywords'를 리스트로 변환
word2vec_data = filtered_channels["target_keywords"].tolist()

# 타겟 채널의 사용된 키워드를 임베딩 데이터에 추가
word2vec_data.append(used_keywords) 

# Word2Vec 모델을 훈련 (vector_size: 벡터 크기, window: 문맥 단어 범위, min_count: 최소 단어 빈도 수, workers: 병렬 처리 수, epochs: 에포크 수)
model = Word2Vec(word2vec_data, vector_size=500, window=5, min_count=1, workers=18, epochs=50)

# 현재 선택된 채널 ID와 일치하지 않는 채널들로 second_results DataFrame 생성
second_results = filtered_channels[(filtered_channels.CHANNEL_ID != channel_id)]

# first_results에 포함된 채널 ID를 제외
second_results = second_results[~second_results.CHANNEL_ID.isin(first_results.CHANNEL_ID.tolist())].copy()

# 인덱스를 재설정하여 0부터 시작하도록 변경
second_results.reset_index(drop=True, inplace=True)

# 사용된 키워드의 벡터를 계산하고 형상을 (1, -1)로 변경
vector1 = get_sentence_vector(used_keywords, model).reshape(1, -1)

# 코사인 유사도를 저장할 딕셔너리 초기화
word2vec_result = dict()

# second_results의 각 행에 대해 반복
for row in second_results.iterrows():
    target_cid = row[1].CHANNEL_ID
    if target_cid != channel_id:
        # 각 채널의 'target_keywords' 벡터 계산
        vector2 = get_sentence_vector(row[1].target_keywords, model).reshape(1, -1)

        # 코사인 유사도 계산
        cos_sim = cosine_similarity(vector1, vector2)[0][0]

        # 유사도가 0보다 큰 경우에만 결과에 추가
        if cos_sim > 0:
            word2vec_result[target_cid] = cos_sim

# 코사인 유사도 기준으로 결과를 정렬 (내림차순)
word2vec_result = dict(sorted(word2vec_result.items(), key=lambda item: item[1], reverse=True))

# word2vec_result의 키에 해당하는 채널들만 second_results에 남김
second_results = second_results[second_results.CHANNEL_ID.isin(word2vec_result.keys())].copy()

# 코사인 유사도를 'score' 열에 추가
second_results['score'] = word2vec_result.values()

second_results.reset_index(drop=True, inplace=True)
second_results

,CHANNEL_ID,CHANNEL_NAME,CHANNEL_DESCRIPTION,CHANNEL_TAGS,MAINLY_USED_KEYWORDS,MAINLY_USED_TAGS,CHANNEL_COUNTRY,CHANNEL_LINK,CHANNEL_SINCE,CHANNEL_CLUSTER,CRAWLED_DATE,USER_ID,channel_id_part,CHANNEL_THUMBNAIL,channel_subscribers,subscribers_cluster,target_keywords,score
0,UC4_AL-HBcYBjCTyNnMw0DOw,자동차읽어주는남자,자동차 읽어주는 남자- 비지니스 문의- 이메일 : lsh6166@naver.com-...,"['bmw', '풀체인지', '시승기', '가격', '페이스리프트', '제네시스',...","[주행, 옵션, 연비, 실내, 차박]","[주행, 옵션, 연비, 트렁크, 실내]",empty,"https://instagram.com/hwee_0721,https://blog.n...",2012-07-16,45,2024-10-13 17:00:19,0.0,_,https://yt3.googleusercontent.com/Sp7kotV0wHkW...,138000,4,"[주행, 옵션, 연비, 실내, 차박, 주행, 옵션, 연비, 트렁크, 실내]",0.112293
1,UCsjzJWuBnPyA8-tj9pEDwXg,오토기어 autogear - 자동차에 대한 모든 정보,오토기어 채널은 대한민국 자동차 전문 채널입니다.\n\n2016년 10월 26일 자...,NaN,"[ESS, 삼상, 오버스티어, 통합, 맥북]","[노이, 클라세, 난연플라스틱, 뱅앤올롭슨, 울트라헤드폰]",대한민국,NaN,2009-07-22,45,2024-10-12 17:00:18,0.0,j,https://yt3.googleusercontent.com/VhUObB1OkZlX...,361000,4,"[ESS, 삼상, 오버스티어, 통합, 맥북, 노이, 클라세, 난연플라스틱, 뱅앤올롭...",0.099823
2,UCzm5GOElgUN-oLutNduDObA,carwow 한국,"Carwow 한국 채널 방문을 환영합니다!​저희는 최고의 자동차 리뷰, 자동차 뉴스...","['review', 'car', 'new', 'bmw', 'best', 'porsc...","[CARWOW, WATSON, MAT, 신형, 레이스]","[드래그, 레이스, 신형, RACING, 람보르기니]",대한민국,"https://www.instagram.com/carwow,https://twitt...",2019-02-22,45,2024-10-13 17:00:19,0.0,m,https://yt3.googleusercontent.com/nHyZXGz4Nb4h...,261000,4,"[CARWOW, WATSON, MAT, 신형, 레이스, 드래그, 레이스, 신형, R...",0.084212
3,UCrnLjvXSPsHxthINSahFwjw,The Mount,The Mount | Car Reviews\n\n안녕하세요 마운트 MOUNT입니다...,NaN,"[시점, REVIEW, VISUAL, 블랙, BLACK]","[화이트, 블랙, 롱레인지, 자동차, 페이스리프트]",대한민국,NaN,2020-05-09,45,2024-10-13 17:00:20,0.0,n,https://yt3.googleusercontent.com/ytc/AIdro_nx...,63800,3,"[시점, REVIEW, VISUAL, 블랙, BLACK, 화이트, 블랙, 롱레인지,...",0.055071
4,UCtZJUcLZq3zmLWjSzwk5phg,Top Gear Magazine Korea,The Official YouTube Channel of BBC Top Gear K...,"['탑기어', '시승기', '시승', '슈퍼카', 'topgear', '스포츠카',...","[UK, RIVIAN, HYUNDAI, R1T]","[UK, RIVIAN, HYUNDAI, R1T]",대한민국,"https://naver.me/G99ObQCj,https://instagram.co...",2019-06-01,45,2024-10-12 17:00:18,0.0,z,https://yt3.googleusercontent.com/KgXEz1MQRp-5...,86900,3,"[UK, RIVIAN, HYUNDAI, R1T, UK, RIVIAN, HYUNDAI...",0.044247


In [16]:
# 상위 N개 결과 출력
ntop = 10

# 첫 번째 결과와 두 번째 결과를 세로로 결합하여 하나의 DataFrame 생성
results = pd.concat([first_results, second_results], axis=0)

# 점수(score)를 기준으로 결과를 내림차순 정렬한 후 상위 N개 결과 출력
results.sort_values(by="score", ascending=False).head(ntop).reset_index(drop=True)


,CHANNEL_ID,CHANNEL_NAME,CHANNEL_DESCRIPTION,CHANNEL_TAGS,MAINLY_USED_KEYWORDS,MAINLY_USED_TAGS,CHANNEL_COUNTRY,CHANNEL_LINK,CHANNEL_SINCE,CHANNEL_CLUSTER,CRAWLED_DATE,USER_ID,channel_id_part,CHANNEL_THUMBNAIL,channel_subscribers,subscribers_cluster,target_keywords,score
0,UCrOl7IuctoHW0iW-Gozj4EQ,비피디 BPD,비피디와 솔님의 자동차 이야기광고 마케팅 문의: bpdbpd.biz@gmail.com,"['솔님', '비피디', '자동차', '시승기', '비린내PD', 'VEHICLE'...","[솔님, 전기차, 시승기, 싼타페, 하이브리드]","[솔님, 하이브리드, 시승기, 싼타페, 전기차]",대한민국,"https://www.instagram.com/bpd.pd/,https://www....",2012-07-27,45,2024-10-13 17:00:19,0.0,o,https://yt3.googleusercontent.com/ytc/AIdro_le...,348000,4,"[솔님, 전기차, 시승기, 싼타페, 하이브리드, 솔님, 하이브리드, 시승기, 싼타페...",1.083333
1,UC4_AL-HBcYBjCTyNnMw0DOw,자동차읽어주는남자,자동차 읽어주는 남자- 비지니스 문의- 이메일 : lsh6166@naver.com-...,"['bmw', '풀체인지', '시승기', '가격', '페이스리프트', '제네시스',...","[주행, 옵션, 연비, 실내, 차박]","[주행, 옵션, 연비, 트렁크, 실내]",empty,"https://instagram.com/hwee_0721,https://blog.n...",2012-07-16,45,2024-10-13 17:00:19,0.0,_,https://yt3.googleusercontent.com/Sp7kotV0wHkW...,138000,4,"[주행, 옵션, 연비, 실내, 차박, 주행, 옵션, 연비, 트렁크, 실내]",0.112293
2,UCsjzJWuBnPyA8-tj9pEDwXg,오토기어 autogear - 자동차에 대한 모든 정보,오토기어 채널은 대한민국 자동차 전문 채널입니다.\n\n2016년 10월 26일 자...,NaN,"[ESS, 삼상, 오버스티어, 통합, 맥북]","[노이, 클라세, 난연플라스틱, 뱅앤올롭슨, 울트라헤드폰]",대한민국,NaN,2009-07-22,45,2024-10-12 17:00:18,0.0,j,https://yt3.googleusercontent.com/VhUObB1OkZlX...,361000,4,"[ESS, 삼상, 오버스티어, 통합, 맥북, 노이, 클라세, 난연플라스틱, 뱅앤올롭...",0.099823
3,UCzm5GOElgUN-oLutNduDObA,carwow 한국,"Carwow 한국 채널 방문을 환영합니다!​저희는 최고의 자동차 리뷰, 자동차 뉴스...","['review', 'car', 'new', 'bmw', 'best', 'porsc...","[CARWOW, WATSON, MAT, 신형, 레이스]","[드래그, 레이스, 신형, RACING, 람보르기니]",대한민국,"https://www.instagram.com/carwow,https://twitt...",2019-02-22,45,2024-10-13 17:00:19,0.0,m,https://yt3.googleusercontent.com/nHyZXGz4Nb4h...,261000,4,"[CARWOW, WATSON, MAT, 신형, 레이스, 드래그, 레이스, 신형, R...",0.084212
4,UCrnLjvXSPsHxthINSahFwjw,The Mount,The Mount | Car Reviews\n\n안녕하세요 마운트 MOUNT입니다...,NaN,"[시점, REVIEW, VISUAL, 블랙, BLACK]","[화이트, 블랙, 롱레인지, 자동차, 페이스리프트]",대한민국,NaN,2020-05-09,45,2024-10-13 17:00:20,0.0,n,https://yt3.googleusercontent.com/ytc/AIdro_nx...,63800,3,"[시점, REVIEW, VISUAL, 블랙, BLACK, 화이트, 블랙, 롱레인지,...",0.055071
5,UCtZJUcLZq3zmLWjSzwk5phg,Top Gear Magazine Korea,The Official YouTube Channel of BBC Top Gear K...,"['탑기어', '시승기', '시승', '슈퍼카', 'topgear', '스포츠카',...","[UK, RIVIAN, HYUNDAI, R1T]","[UK, RIVIAN, HYUNDAI, R1T]",대한민국,"https://naver.me/G99ObQCj,https://instagram.co...",2019-06-01,45,2024-10-12 17:00:18,0.0,z,https://yt3.googleusercontent.com/KgXEz1MQRp-5...,86900,3,"[UK, RIVIAN, HYUNDAI, R1T, UK, RIVIAN, HYUNDAI...",0.044247
